# Summary:
- Only the abstracts are used to get the vectors
- Experiment results:

    - SVM 
                         precision    recall  f1-score   support

            0.0               0.77      0.81      0.79      2590
            1.0               0.76      0.71      0.74      2188

            avg / total       0.77      0.77      0.77      4778

            Confusion Matrix
            2109  481
            625  1563

    - DNN + SVM
                         precision    recall  f1-score   support

            0.0               0.74      0.82      0.78      2590
            1.0               0.75      0.66      0.70      2188

            avg / total       0.74      0.74      0.74      4778

            Confusion Matrix
            2112  478
            746  1442
               
    - Deeper DNN with dropout + SVM
                         precision    recall  f1-score   support

            0.0               0.77      0.78      0.78      2590
            1.0               0.74      0.72      0.73      2188

            avg / total       0.75      0.75      0.75      4778

            Confusion Matrix        
            2021  569
             604 1584
    
    - Inception Batch Normalization + SVM
                         precision    recall  f1-score   support

            0.0               0.75      0.78      0.76      2590
            1.0               0.73      0.68      0.70      2188

            avg / total       0.74      0.74      0.74      4778
        
            Confusion Matrix
            2031  559
             695 1493

In [1]:
import glob
import multiprocessing
import time
import csv
import pickle
import os
import pandas as pd
import shutil
import numpy as np
import warnings
import tqdm
import logging
import mxnet as mx
from bs4 import BeautifulSoup
from urllib.request import urlopen
from os.path import basename
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

%pylab inline
warnings.filterwarnings("ignore")

/home/dc/anaconda3/envs/mxnet/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


Populating the interactive namespace from numpy and matplotlib


In [2]:
# 2d to 4d conversion
def to4d(img):
    return img.reshape(img.shape[0], 1, 32, 32).astype(np.float32)

In [3]:
def modelMetric(mod, info, nd=True):
    if nd == True: 
        prob = mod.predict(val_iters).asnumpy()
    else:
        prob = mod.predict(val_iters)
    logging.info(info)
    # Check the accuracy from prediction
    val_iters.reset()

    y_batch = []
    for dbatch in val_iters:
        label = dbatch.label[0].asnumpy()
        pad = val_iters.getpad()
        real_size = label.shape[0] - pad
        y_batch.append(label[0:real_size])
    y = np.concatenate(y_batch)

    # get prediction label 
    py = np.argmax(prob, axis=1)
    acc1 = sum(py == y) / len(y)
    logging.info('Test accuracy = %f', acc1)
    cfn_matrix=confusion_matrix(y,py)
    print(classification_report (y,py))
    print(cfn_matrix)

# 1. Data Preparation

In [4]:
p_file = 'RCT_Vectors'
with open(p_file, 'rb') as fin:
    vectors = pickle.load(fin)

In [5]:
p_file = 'RCT_labels'
with open(p_file, 'rb') as fin:
    data_Y = pickle.load(fin)

In [6]:
d2v_X_train, d2v_X_test, d2v_Y_train, d2v_Y_test = train_test_split(vectors, data_Y, test_size=0.25)

In [7]:
X_train_array = np.array(d2v_X_train)
Y_train_array = np.array(d2v_Y_train)
X_test_array = np.array(d2v_X_test)
Y_test_array = np.array(d2v_Y_test)

In [8]:
# double check whether oversampling works
X_train_array.shape, Y_train_array.shape, X_test_array.shape, Y_test_array.shape, len(vectors)

((14333, 1024), (14333,), (4778, 1024), (4778,), 19111)

In [9]:
# creating 4d training/testing dataset
batch_size = 64
train_iters = mx.io.NDArrayIter(to4d(X_train_array), Y_train_array, batch_size, shuffle=True,label_name='svm_label')
val_iters = mx.io.NDArrayIter(to4d(X_test_array), Y_test_array, batch_size,label_name='svm_label')

# 2. Train models
## 2.1 SVM

In [10]:
# Single SVM
data = mx.symbol.Variable('data')
svm  = mx.symbol.FullyConnected(data = data, name='fc', num_hidden=2)

# Here we add the ultimate layer based on L2-SVM objective
ssvm = mx.symbol.SVMOutput(data=svm, name='svm')
# draw network chart
shape = {"data" : (batch_size, 1, 32, 32)}
mx.viz.plot_network(symbol=ssvm, shape=shape)
network_chart=mx.viz.plot_network(symbol=ssvm, shape=shape)
network_chart.format = 'png'
network_chart.render('SVM')

'SVM.png'

<img src='SVM.png'>

In [11]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

mod = mx.model.FeedForward(
    ctx = mx.gpu(0),     # use GPU 0 for training, others are same as before
    symbol = ssvm,       # network structure
    num_epoch = 300,     # number of data passes for training 
    learning_rate = 0.01 # learning rate of SGD 
)
mod.fit(
    X=train_iters,       # training data
    eval_data=val_iters, # validation data
    batch_end_callback = mx.callback.Speedometer(batch_size, batch_size) # output progress for each 200 data batches
)


INFO:root:Start training with [gpu(0)]
INFO:root:Epoch[0] Batch [64]	Speed: 60605.39 samples/sec	accuracy=0.710205
INFO:root:Epoch[0] Batch [128]	Speed: 58303.25 samples/sec	accuracy=0.764404
INFO:root:Epoch[0] Batch [192]	Speed: 55004.45 samples/sec	accuracy=0.782227
INFO:root:Epoch[0] Resetting Data Iterator
INFO:root:Epoch[0] Time cost=0.263
INFO:root:Epoch[0] Validation-accuracy=0.761667
INFO:root:Epoch[1] Batch [64]	Speed: 58607.56 samples/sec	accuracy=0.774414
INFO:root:Epoch[1] Batch [128]	Speed: 67275.74 samples/sec	accuracy=0.780518
INFO:root:Epoch[1] Batch [192]	Speed: 65914.68 samples/sec	accuracy=0.792969
INFO:root:Epoch[1] Resetting Data Iterator
INFO:root:Epoch[1] Time cost=0.230
INFO:root:Epoch[1] Validation-accuracy=0.767500
INFO:root:Epoch[2] Batch [64]	Speed: 66961.60 samples/sec	accuracy=0.779053
INFO:root:Epoch[2] Batch [128]	Speed: 61220.24 samples/sec	accuracy=0.784424
INFO:root:Epoch[2] Batch [192]	Speed: 62254.02 samples/sec	accuracy=0.797607
INFO:root:Epoch[2] 

INFO:root:Epoch[22] Time cost=0.207
INFO:root:Epoch[22] Validation-accuracy=0.772500
INFO:root:Epoch[23] Batch [64]	Speed: 68565.07 samples/sec	accuracy=0.791748
INFO:root:Epoch[23] Batch [128]	Speed: 69239.88 samples/sec	accuracy=0.793457
INFO:root:Epoch[23] Batch [192]	Speed: 68761.01 samples/sec	accuracy=0.799805
INFO:root:Epoch[23] Resetting Data Iterator
INFO:root:Epoch[23] Time cost=0.218
INFO:root:Epoch[23] Validation-accuracy=0.772708
INFO:root:Epoch[24] Batch [64]	Speed: 66102.10 samples/sec	accuracy=0.791748
INFO:root:Epoch[24] Batch [128]	Speed: 64162.65 samples/sec	accuracy=0.793701
INFO:root:Epoch[24] Batch [192]	Speed: 68568.08 samples/sec	accuracy=0.800049
INFO:root:Epoch[24] Resetting Data Iterator
INFO:root:Epoch[24] Time cost=0.223
INFO:root:Epoch[24] Validation-accuracy=0.772917
INFO:root:Epoch[25] Batch [64]	Speed: 67707.39 samples/sec	accuracy=0.792236
INFO:root:Epoch[25] Batch [128]	Speed: 67188.65 samples/sec	accuracy=0.793701
INFO:root:Epoch[25] Batch [192]	Spee

INFO:root:Epoch[67] Validation-accuracy=0.769792
INFO:root:Epoch[68] Batch [64]	Speed: 75541.81 samples/sec	accuracy=0.791504
INFO:root:Epoch[68] Batch [128]	Speed: 72081.96 samples/sec	accuracy=0.795654
INFO:root:Epoch[68] Batch [192]	Speed: 69125.38 samples/sec	accuracy=0.798828
INFO:root:Epoch[68] Resetting Data Iterator
INFO:root:Epoch[68] Time cost=0.210
INFO:root:Epoch[68] Validation-accuracy=0.770000
INFO:root:Epoch[69] Batch [64]	Speed: 75325.86 samples/sec	accuracy=0.791748
INFO:root:Epoch[69] Batch [128]	Speed: 66434.66 samples/sec	accuracy=0.795654
INFO:root:Epoch[69] Batch [192]	Speed: 64930.89 samples/sec	accuracy=0.799072
INFO:root:Epoch[69] Resetting Data Iterator
INFO:root:Epoch[69] Time cost=0.216
INFO:root:Epoch[69] Validation-accuracy=0.770208
INFO:root:Epoch[70] Batch [64]	Speed: 67996.54 samples/sec	accuracy=0.792236
INFO:root:Epoch[70] Batch [128]	Speed: 68068.47 samples/sec	accuracy=0.795654
INFO:root:Epoch[70] Batch [192]	Speed: 65348.54 samples/sec	accuracy=0.7

INFO:root:Epoch[90] Resetting Data Iterator
INFO:root:Epoch[90] Time cost=0.200
INFO:root:Epoch[90] Validation-accuracy=0.769792
INFO:root:Epoch[91] Batch [64]	Speed: 68617.37 samples/sec	accuracy=0.794678
INFO:root:Epoch[91] Batch [128]	Speed: 72327.76 samples/sec	accuracy=0.796143
INFO:root:Epoch[91] Batch [192]	Speed: 69301.61 samples/sec	accuracy=0.800049
INFO:root:Epoch[91] Resetting Data Iterator
INFO:root:Epoch[91] Time cost=0.211
INFO:root:Epoch[91] Validation-accuracy=0.769792
INFO:root:Epoch[92] Batch [64]	Speed: 67866.53 samples/sec	accuracy=0.794922
INFO:root:Epoch[92] Batch [128]	Speed: 69493.64 samples/sec	accuracy=0.796387
INFO:root:Epoch[92] Batch [192]	Speed: 73019.76 samples/sec	accuracy=0.799805
INFO:root:Epoch[92] Resetting Data Iterator
INFO:root:Epoch[92] Time cost=0.206
INFO:root:Epoch[92] Validation-accuracy=0.769583
INFO:root:Epoch[93] Batch [64]	Speed: 71474.68 samples/sec	accuracy=0.794922
INFO:root:Epoch[93] Batch [128]	Speed: 68285.73 samples/sec	accuracy=0

INFO:root:Epoch[113] Batch [128]	Speed: 80561.73 samples/sec	accuracy=0.795166
INFO:root:Epoch[113] Batch [192]	Speed: 82584.80 samples/sec	accuracy=0.800537
INFO:root:Epoch[113] Resetting Data Iterator
INFO:root:Epoch[113] Time cost=0.188
INFO:root:Epoch[113] Validation-accuracy=0.769375
INFO:root:Epoch[114] Batch [64]	Speed: 76583.70 samples/sec	accuracy=0.796631
INFO:root:Epoch[114] Batch [128]	Speed: 78969.02 samples/sec	accuracy=0.794922
INFO:root:Epoch[114] Batch [192]	Speed: 66561.55 samples/sec	accuracy=0.800537
INFO:root:Epoch[114] Resetting Data Iterator
INFO:root:Epoch[114] Time cost=0.203
INFO:root:Epoch[114] Validation-accuracy=0.769167
INFO:root:Epoch[115] Batch [64]	Speed: 70454.92 samples/sec	accuracy=0.796387
INFO:root:Epoch[115] Batch [128]	Speed: 70817.37 samples/sec	accuracy=0.794922
INFO:root:Epoch[115] Batch [192]	Speed: 67348.27 samples/sec	accuracy=0.800781
INFO:root:Epoch[115] Resetting Data Iterator
INFO:root:Epoch[115] Time cost=0.212
INFO:root:Epoch[115] Val

...

INFO:root:Epoch[202] Time cost=0.214
INFO:root:Epoch[202] Validation-accuracy=0.769375
INFO:root:Epoch[203] Batch [64]	Speed: 70493.95 samples/sec	accuracy=0.796875
INFO:root:Epoch[203] Batch [128]	Speed: 67612.52 samples/sec	accuracy=0.796143
INFO:root:Epoch[203] Batch [192]	Speed: 70612.46 samples/sec	accuracy=0.801514
INFO:root:Epoch[203] Resetting Data Iterator
INFO:root:Epoch[203] Time cost=0.213
INFO:root:Epoch[203] Validation-accuracy=0.769375
INFO:root:Epoch[204] Batch [64]	Speed: 74468.12 samples/sec	accuracy=0.796875
INFO:root:Epoch[204] Batch [128]	Speed: 71725.94 samples/sec	accuracy=0.796387
INFO:root:Epoch[204] Batch [192]	Speed: 81017.63 samples/sec	accuracy=0.801514
INFO:root:Epoch[204] Resetting Data Iterator
INFO:root:Epoch[204] Time cost=0.195
INFO:root:Epoch[204] Validation-accuracy=0.769375
INFO:root:Epoch[205] Batch [64]	Speed: 77437.03 samples/sec	accuracy=0.796875
INFO:root:Epoch[205] Batch [128]	Speed: 72298.54 samples/sec	accuracy=0.796387
INFO:root:Epoch[205]

INFO:root:Epoch[225] Batch [128]	Speed: 69130.11 samples/sec	accuracy=0.796143
INFO:root:Epoch[225] Batch [192]	Speed: 76497.09 samples/sec	accuracy=0.801758
INFO:root:Epoch[225] Resetting Data Iterator
INFO:root:Epoch[225] Time cost=0.204
INFO:root:Epoch[225] Validation-accuracy=0.769375
INFO:root:Epoch[226] Batch [64]	Speed: 70302.69 samples/sec	accuracy=0.797119
INFO:root:Epoch[226] Batch [128]	Speed: 70807.15 samples/sec	accuracy=0.796143
INFO:root:Epoch[226] Batch [192]	Speed: 75724.94 samples/sec	accuracy=0.802002
INFO:root:Epoch[226] Resetting Data Iterator
INFO:root:Epoch[226] Time cost=0.202
INFO:root:Epoch[226] Validation-accuracy=0.769167
INFO:root:Epoch[227] Batch [64]	Speed: 72634.78 samples/sec	accuracy=0.797119
INFO:root:Epoch[227] Batch [128]	Speed: 73440.53 samples/sec	accuracy=0.796143
INFO:root:Epoch[227] Batch [192]	Speed: 68923.49 samples/sec	accuracy=0.802002
INFO:root:Epoch[227] Resetting Data Iterator
INFO:root:Epoch[227] Time cost=0.206
INFO:root:Epoch[227] Val

INFO:root:Epoch[247] Resetting Data Iterator
INFO:root:Epoch[247] Time cost=0.194
INFO:root:Epoch[247] Validation-accuracy=0.769167
INFO:root:Epoch[248] Batch [64]	Speed: 83091.68 samples/sec	accuracy=0.796875
INFO:root:Epoch[248] Batch [128]	Speed: 73379.99 samples/sec	accuracy=0.796631
INFO:root:Epoch[248] Batch [192]	Speed: 72516.13 samples/sec	accuracy=0.803467
INFO:root:Epoch[248] Resetting Data Iterator
INFO:root:Epoch[248] Time cost=0.201
INFO:root:Epoch[248] Validation-accuracy=0.769167
INFO:root:Epoch[249] Batch [64]	Speed: 65522.75 samples/sec	accuracy=0.797119
INFO:root:Epoch[249] Batch [128]	Speed: 66719.49 samples/sec	accuracy=0.796631
INFO:root:Epoch[249] Batch [192]	Speed: 68569.17 samples/sec	accuracy=0.803223
INFO:root:Epoch[249] Resetting Data Iterator
INFO:root:Epoch[249] Time cost=0.220
INFO:root:Epoch[249] Validation-accuracy=0.768958
INFO:root:Epoch[250] Batch [64]	Speed: 69025.40 samples/sec	accuracy=0.797119
INFO:root:Epoch[250] Batch [128]	Speed: 66087.86 sampl

INFO:root:Epoch[270] Batch [64]	Speed: 73397.54 samples/sec	accuracy=0.797119
INFO:root:Epoch[270] Batch [128]	Speed: 70266.75 samples/sec	accuracy=0.796875
INFO:root:Epoch[270] Batch [192]	Speed: 78459.07 samples/sec	accuracy=0.804199
INFO:root:Epoch[270] Resetting Data Iterator
INFO:root:Epoch[270] Time cost=0.199
INFO:root:Epoch[270] Validation-accuracy=0.768958
INFO:root:Epoch[271] Batch [64]	Speed: 77551.68 samples/sec	accuracy=0.797119
INFO:root:Epoch[271] Batch [128]	Speed: 76760.61 samples/sec	accuracy=0.796875
INFO:root:Epoch[271] Batch [192]	Speed: 77374.95 samples/sec	accuracy=0.804199
INFO:root:Epoch[271] Resetting Data Iterator
INFO:root:Epoch[271] Time cost=0.194
INFO:root:Epoch[271] Validation-accuracy=0.768750
INFO:root:Epoch[272] Batch [64]	Speed: 74882.29 samples/sec	accuracy=0.797119
INFO:root:Epoch[272] Batch [128]	Speed: 74808.27 samples/sec	accuracy=0.796875
INFO:root:Epoch[272] Batch [192]	Speed: 68603.13 samples/sec	accuracy=0.803955
INFO:root:Epoch[272] Resetti

INFO:root:Epoch[292] Batch [192]	Speed: 64633.11 samples/sec	accuracy=0.803223
INFO:root:Epoch[292] Resetting Data Iterator
INFO:root:Epoch[292] Time cost=0.222
INFO:root:Epoch[292] Validation-accuracy=0.768333
INFO:root:Epoch[293] Batch [64]	Speed: 71282.21 samples/sec	accuracy=0.797363
INFO:root:Epoch[293] Batch [128]	Speed: 72212.85 samples/sec	accuracy=0.797363
INFO:root:Epoch[293] Batch [192]	Speed: 64211.09 samples/sec	accuracy=0.803223
INFO:root:Epoch[293] Resetting Data Iterator
INFO:root:Epoch[293] Time cost=0.214
INFO:root:Epoch[293] Validation-accuracy=0.768333
INFO:root:Epoch[294] Batch [64]	Speed: 67063.54 samples/sec	accuracy=0.797363
INFO:root:Epoch[294] Batch [128]	Speed: 66741.01 samples/sec	accuracy=0.797607
INFO:root:Epoch[294] Batch [192]	Speed: 65004.10 samples/sec	accuracy=0.803223
INFO:root:Epoch[294] Resetting Data Iterator
INFO:root:Epoch[294] Time cost=0.228
INFO:root:Epoch[294] Validation-accuracy=0.768333
INFO:root:Epoch[295] Batch [64]	Speed: 65580.03 sampl

In [12]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'SVM'
epoch = 300
mod.save(prefix, epoch)

INFO:root:Saved checkpoint to "SVM-0100.params"


In [13]:
modelMetric(mod, 'Finish predict of one Layer SVM Network...', False)

INFO:root:Finish predict of one Layer SVM Network...
INFO:root:Test accuracy = 0.768522


             precision    recall  f1-score   support

        0.0       0.77      0.81      0.79      2590
        1.0       0.76      0.71      0.74      2188

avg / total       0.77      0.77      0.77      4778

[[2109  481]
 [ 625 1563]]


## 2.2 DNN + SVM

In [14]:
# DNN + SVM 
data = mx.symbol.Variable('data')
data = mx.sym.Flatten(data=data)

fc1  = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=512)
act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
fc2  = mx.symbol.FullyConnected(data = act1, name = 'fc2', num_hidden = 512)
act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")

fc3  = mx.symbol.FullyConnected(data = act2, name='fc3', num_hidden=2)

dnnsvm = mx.symbol.SVMOutput(data=fc3, name='svm')

# draw network chart
shape = {"data" : (batch_size, 1, 32, 32)}
mx.viz.plot_network(symbol=dnnsvm, shape=shape)
network_chart=mx.viz.plot_network(symbol=dnnsvm, shape=shape)
network_chart.format = 'png'
network_chart.render('DNN + SVM')

'DNN + SVM.png'

<img src='DNN + SVM.png'>

In [15]:
mod = mx.mod.Module(
    context = mx.gpu(0),  # Run on GPU 0
    symbol = dnnsvm,        
    label_names = ['svm_label'],
)
mod.fit(
    train_data=train_iters,
    eval_data=val_iters,  
    batch_end_callback = mx.callback.Speedometer(batch_size, batch_size), 
    num_epoch = 300,       # Train for 100 epochs
    optimizer_params = {
        'learning_rate': 0.001,  # Learning rate
        'momentum': 0.9,       # Momentum for SGD with momentum
        'wd': 0.001,         # Weight decay for regularization
    },
)
print('Accuracy:', mod.score(val_iters, mx.metric.Accuracy())[0][1]*100, '%')

INFO:root:Epoch[0] Batch [64]	Speed: 48234.35 samples/sec	accuracy=0.553846
INFO:root:Epoch[0] Batch [128]	Speed: 47971.58 samples/sec	accuracy=0.553467
INFO:root:Epoch[0] Batch [192]	Speed: 51930.85 samples/sec	accuracy=0.552002
INFO:root:Epoch[0] Train-accuracy=0.555444
INFO:root:Epoch[0] Time cost=0.298
INFO:root:Epoch[0] Validation-accuracy=0.542292
INFO:root:Epoch[1] Batch [64]	Speed: 48131.38 samples/sec	accuracy=0.553846
INFO:root:Epoch[1] Batch [128]	Speed: 40863.00 samples/sec	accuracy=0.553467
INFO:root:Epoch[1] Batch [192]	Speed: 39911.05 samples/sec	accuracy=0.552002
INFO:root:Epoch[1] Train-accuracy=0.555444
INFO:root:Epoch[1] Time cost=0.347
INFO:root:Epoch[1] Validation-accuracy=0.542292
INFO:root:Epoch[2] Batch [64]	Speed: 51029.72 samples/sec	accuracy=0.553846
INFO:root:Epoch[2] Batch [128]	Speed: 51748.15 samples/sec	accuracy=0.553467
INFO:root:Epoch[2] Batch [192]	Speed: 52314.65 samples/sec	accuracy=0.552002
INFO:root:Epoch[2] Train-accuracy=0.555444
INFO:root:Epoch

INFO:root:Epoch[22] Validation-accuracy=0.783125
INFO:root:Epoch[23] Batch [64]	Speed: 50580.65 samples/sec	accuracy=0.841346
INFO:root:Epoch[23] Batch [128]	Speed: 49560.70 samples/sec	accuracy=0.850586
INFO:root:Epoch[23] Batch [192]	Speed: 49835.44 samples/sec	accuracy=0.851562
INFO:root:Epoch[23] Train-accuracy=0.852319
INFO:root:Epoch[23] Time cost=0.291
INFO:root:Epoch[23] Validation-accuracy=0.783542
INFO:root:Epoch[24] Batch [64]	Speed: 54499.65 samples/sec	accuracy=0.850481
INFO:root:Epoch[24] Batch [128]	Speed: 49386.31 samples/sec	accuracy=0.859619
INFO:root:Epoch[24] Batch [192]	Speed: 48223.79 samples/sec	accuracy=0.862549
INFO:root:Epoch[24] Train-accuracy=0.859375
INFO:root:Epoch[24] Time cost=0.290
INFO:root:Epoch[24] Validation-accuracy=0.781875
INFO:root:Epoch[25] Batch [64]	Speed: 49187.65 samples/sec	accuracy=0.864663
INFO:root:Epoch[25] Batch [128]	Speed: 39818.54 samples/sec	accuracy=0.868896
INFO:root:Epoch[25] Batch [192]	Speed: 43446.15 samples/sec	accuracy=0.8

INFO:root:Epoch[45] Train-accuracy=0.974798
INFO:root:Epoch[45] Time cost=0.313
INFO:root:Epoch[45] Validation-accuracy=0.747083
INFO:root:Epoch[46] Batch [64]	Speed: 53345.51 samples/sec	accuracy=0.959615
INFO:root:Epoch[46] Batch [128]	Speed: 48185.38 samples/sec	accuracy=0.963135
INFO:root:Epoch[46] Batch [192]	Speed: 55013.43 samples/sec	accuracy=0.982178
INFO:root:Epoch[46] Train-accuracy=0.981855
INFO:root:Epoch[46] Time cost=0.282
INFO:root:Epoch[46] Validation-accuracy=0.751667
INFO:root:Epoch[47] Batch [64]	Speed: 54736.14 samples/sec	accuracy=0.975481
INFO:root:Epoch[47] Batch [128]	Speed: 50820.05 samples/sec	accuracy=0.959229
INFO:root:Epoch[47] Batch [192]	Speed: 46389.45 samples/sec	accuracy=0.981689
INFO:root:Epoch[47] Train-accuracy=0.983871
INFO:root:Epoch[47] Time cost=0.298
INFO:root:Epoch[47] Validation-accuracy=0.755417
INFO:root:Epoch[48] Batch [64]	Speed: 50226.93 samples/sec	accuracy=0.982452
INFO:root:Epoch[48] Batch [128]	Speed: 46749.47 samples/sec	accuracy=0

INFO:root:Epoch[68] Batch [192]	Speed: 48823.37 samples/sec	accuracy=0.989990
INFO:root:Epoch[68] Train-accuracy=0.989415
INFO:root:Epoch[68] Time cost=0.318
INFO:root:Epoch[68] Validation-accuracy=0.754583
INFO:root:Epoch[69] Batch [64]	Speed: 42619.90 samples/sec	accuracy=0.991827
INFO:root:Epoch[69] Batch [128]	Speed: 46146.10 samples/sec	accuracy=0.991455
INFO:root:Epoch[69] Batch [192]	Speed: 48692.88 samples/sec	accuracy=0.990479
INFO:root:Epoch[69] Train-accuracy=0.989415
INFO:root:Epoch[69] Time cost=0.317
INFO:root:Epoch[69] Validation-accuracy=0.754792
INFO:root:Epoch[70] Batch [64]	Speed: 42179.17 samples/sec	accuracy=0.992067
INFO:root:Epoch[70] Batch [128]	Speed: 46256.55 samples/sec	accuracy=0.991455
INFO:root:Epoch[70] Batch [192]	Speed: 49394.41 samples/sec	accuracy=0.990723
INFO:root:Epoch[70] Train-accuracy=0.989919
INFO:root:Epoch[70] Time cost=0.316
INFO:root:Epoch[70] Validation-accuracy=0.753750
INFO:root:Epoch[71] Batch [64]	Speed: 52374.62 samples/sec	accuracy=0

INFO:root:Epoch[91] Batch [128]	Speed: 47251.96 samples/sec	accuracy=0.994385
INFO:root:Epoch[91] Batch [192]	Speed: 48447.36 samples/sec	accuracy=0.995361
INFO:root:Epoch[91] Train-accuracy=0.992944
INFO:root:Epoch[91] Time cost=0.315
INFO:root:Epoch[91] Validation-accuracy=0.749792
INFO:root:Epoch[92] Batch [64]	Speed: 42228.83 samples/sec	accuracy=0.995192
INFO:root:Epoch[92] Batch [128]	Speed: 50689.00 samples/sec	accuracy=0.994629
INFO:root:Epoch[92] Batch [192]	Speed: 55064.92 samples/sec	accuracy=0.995605
INFO:root:Epoch[92] Train-accuracy=0.992944
INFO:root:Epoch[92] Time cost=0.296
INFO:root:Epoch[92] Validation-accuracy=0.753542
INFO:root:Epoch[93] Batch [64]	Speed: 41732.85 samples/sec	accuracy=0.995192
INFO:root:Epoch[93] Batch [128]	Speed: 48299.98 samples/sec	accuracy=0.994629
INFO:root:Epoch[93] Batch [192]	Speed: 49528.41 samples/sec	accuracy=0.995850
INFO:root:Epoch[93] Train-accuracy=0.993448
INFO:root:Epoch[93] Time cost=0.314
INFO:root:Epoch[93] Validation-accuracy=

...

INFO:root:Epoch[225] Train-accuracy=0.999496
INFO:root:Epoch[225] Time cost=0.311
INFO:root:Epoch[225] Validation-accuracy=0.745000
INFO:root:Epoch[226] Batch [64]	Speed: 42273.82 samples/sec	accuracy=0.999519
INFO:root:Epoch[226] Batch [128]	Speed: 47855.19 samples/sec	accuracy=0.999023
INFO:root:Epoch[226] Batch [192]	Speed: 48765.02 samples/sec	accuracy=0.999756
INFO:root:Epoch[226] Train-accuracy=0.999496
INFO:root:Epoch[226] Time cost=0.315
INFO:root:Epoch[226] Validation-accuracy=0.745625
INFO:root:Epoch[227] Batch [64]	Speed: 46001.78 samples/sec	accuracy=0.999519
INFO:root:Epoch[227] Batch [128]	Speed: 44985.26 samples/sec	accuracy=0.999023
INFO:root:Epoch[227] Batch [192]	Speed: 48977.87 samples/sec	accuracy=0.999756
INFO:root:Epoch[227] Train-accuracy=0.999496
INFO:root:Epoch[227] Time cost=0.311
INFO:root:Epoch[227] Validation-accuracy=0.745833
INFO:root:Epoch[228] Batch [64]	Speed: 41633.33 samples/sec	accuracy=0.999519
INFO:root:Epoch[228] Batch [128]	Speed: 50414.85 sampl

INFO:root:Epoch[248] Batch [64]	Speed: 41762.98 samples/sec	accuracy=0.999519
INFO:root:Epoch[248] Batch [128]	Speed: 45282.99 samples/sec	accuracy=0.999268
INFO:root:Epoch[248] Batch [192]	Speed: 49509.28 samples/sec	accuracy=0.999756
INFO:root:Epoch[248] Train-accuracy=0.999496
INFO:root:Epoch[248] Time cost=0.318
INFO:root:Epoch[248] Validation-accuracy=0.745417
INFO:root:Epoch[249] Batch [64]	Speed: 43777.73 samples/sec	accuracy=0.999519
INFO:root:Epoch[249] Batch [128]	Speed: 45483.68 samples/sec	accuracy=0.999268
INFO:root:Epoch[249] Batch [192]	Speed: 47145.25 samples/sec	accuracy=0.999756
INFO:root:Epoch[249] Train-accuracy=1.000000
INFO:root:Epoch[249] Time cost=0.318
INFO:root:Epoch[249] Validation-accuracy=0.745833
INFO:root:Epoch[250] Batch [64]	Speed: 41994.20 samples/sec	accuracy=0.999519
INFO:root:Epoch[250] Batch [128]	Speed: 46626.14 samples/sec	accuracy=0.999268
INFO:root:Epoch[250] Batch [192]	Speed: 49406.34 samples/sec	accuracy=0.999756
INFO:root:Epoch[250] Train-a

INFO:root:Epoch[270] Batch [192]	Speed: 53116.26 samples/sec	accuracy=0.999756
INFO:root:Epoch[270] Train-accuracy=1.000000
INFO:root:Epoch[270] Time cost=0.312
INFO:root:Epoch[270] Validation-accuracy=0.746042
INFO:root:Epoch[271] Batch [64]	Speed: 41732.85 samples/sec	accuracy=0.999519
INFO:root:Epoch[271] Batch [128]	Speed: 49444.73 samples/sec	accuracy=0.999268
INFO:root:Epoch[271] Batch [192]	Speed: 52557.76 samples/sec	accuracy=0.999756
INFO:root:Epoch[271] Train-accuracy=1.000000
INFO:root:Epoch[271] Time cost=0.306
INFO:root:Epoch[271] Validation-accuracy=0.744792
INFO:root:Epoch[272] Batch [64]	Speed: 41403.66 samples/sec	accuracy=0.999519
INFO:root:Epoch[272] Batch [128]	Speed: 48163.36 samples/sec	accuracy=0.999268
INFO:root:Epoch[272] Batch [192]	Speed: 53955.52 samples/sec	accuracy=0.999756
INFO:root:Epoch[272] Train-accuracy=1.000000
INFO:root:Epoch[272] Time cost=0.306
INFO:root:Epoch[272] Validation-accuracy=0.744375
INFO:root:Epoch[273] Batch [64]	Speed: 41546.34 sampl

INFO:root:Epoch[292] Time cost=0.320
INFO:root:Epoch[292] Validation-accuracy=0.746042
INFO:root:Epoch[293] Batch [64]	Speed: 41409.15 samples/sec	accuracy=0.999519
INFO:root:Epoch[293] Batch [128]	Speed: 48502.89 samples/sec	accuracy=0.999512
INFO:root:Epoch[293] Batch [192]	Speed: 43357.67 samples/sec	accuracy=0.999756
INFO:root:Epoch[293] Train-accuracy=1.000000
INFO:root:Epoch[293] Time cost=0.327
INFO:root:Epoch[293] Validation-accuracy=0.740417
INFO:root:Epoch[294] Batch [64]	Speed: 47007.73 samples/sec	accuracy=0.999519
INFO:root:Epoch[294] Batch [128]	Speed: 45009.65 samples/sec	accuracy=0.999512
INFO:root:Epoch[294] Batch [192]	Speed: 50100.96 samples/sec	accuracy=0.999756
INFO:root:Epoch[294] Train-accuracy=1.000000
INFO:root:Epoch[294] Time cost=0.312
INFO:root:Epoch[294] Validation-accuracy=0.746875
INFO:root:Epoch[295] Batch [64]	Speed: 42665.94 samples/sec	accuracy=0.999519
INFO:root:Epoch[295] Batch [128]	Speed: 45746.74 samples/sec	accuracy=0.999268
INFO:root:Epoch[295]

Accuracy: 74.3125 %


In [16]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'DNN_SVM'
epoch = 300
mod.save_checkpoint(prefix, epoch)

INFO:root:Saved checkpoint to "DNN_SVM-0300.params"


In [17]:
modelMetric(mod, 'Finish predict of DNN+SVM...')

INFO:root:Finish predict of DNN+SVM...
INFO:root:Test accuracy = 0.743826


             precision    recall  f1-score   support

        0.0       0.74      0.82      0.78      2590
        1.0       0.75      0.66      0.70      2188

avg / total       0.74      0.74      0.74      4778

[[2112  478]
 [ 746 1442]]


## 2.3 Deeper DNN with dropout + SVM

In [18]:
# Deeper DNN with dropout + SVM
data = mx.symbol.Variable('data')
data = mx.sym.Flatten(data=data)

fc1  = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=1024)
act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
fc2  = mx.symbol.FullyConnected(data = act1, name = 'fc2', num_hidden = 1024)
act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")
drop1 = mx.sym.Dropout(data=act2, p=0.5)

fc3  = mx.symbol.FullyConnected(data = drop1, name = 'fc3', num_hidden = 512)
act3 = mx.symbol.Activation(data = fc3, name='relu3', act_type="relu")
fc4  = mx.symbol.FullyConnected(data = act3, name = 'fc4', num_hidden = 512)
act4 = mx.symbol.Activation(data = fc4, name='relu4', act_type="relu")

drop2 = mx.sym.Dropout(data=act4, p=0.5)

fc5  = mx.symbol.FullyConnected(data = drop2, name='fc5', num_hidden=2)

ddnn = mx.symbol.SVMOutput(data=fc5, name='svm')

# draw network chart
shape = {"data" : (batch_size, 1, 32, 32)}
mx.viz.plot_network(symbol=ddnn, shape=shape)
network_chart=mx.viz.plot_network(symbol=ddnn, shape=shape)
network_chart.format = 'png'
network_chart.render('DDNN + SVM')

'DDNN + SVM.png'

<img src='DDNN + SVM.png'>

In [19]:
mod = mx.mod.Module(
    context = mx.gpu(0),  # Run on GPU 0
    symbol = ddnn,         # Use the network we just defined
    label_names = ['svm_label'],
)
mod.fit(
    train_data=train_iters,
    eval_data=val_iters,  
    batch_end_callback = mx.callback.Speedometer(batch_size, batch_size),  
    num_epoch = 300,       # Train for 100 epochs
    optimizer_params = {
        'learning_rate': 0.001,  # Learning rate
        'momentum': 0.9,       # Momentum for SGD with momentum
        'wd': 0.00001,         # Weight decay for regularization
    },
)

print('Accuracy:', mod.score(val_iters, mx.metric.Accuracy())[0][1]*100, '%')

INFO:root:Epoch[0] Batch [64]	Speed: 29845.59 samples/sec	accuracy=0.552163
INFO:root:Epoch[0] Batch [128]	Speed: 32823.34 samples/sec	accuracy=0.553467
INFO:root:Epoch[0] Batch [192]	Speed: 35191.45 samples/sec	accuracy=0.552002
INFO:root:Epoch[0] Train-accuracy=0.555444
INFO:root:Epoch[0] Time cost=0.469
INFO:root:Epoch[0] Validation-accuracy=0.542292
INFO:root:Epoch[1] Batch [64]	Speed: 31506.97 samples/sec	accuracy=0.553846
INFO:root:Epoch[1] Batch [128]	Speed: 34655.07 samples/sec	accuracy=0.553467
INFO:root:Epoch[1] Batch [192]	Speed: 35328.30 samples/sec	accuracy=0.552002
INFO:root:Epoch[1] Train-accuracy=0.555444
INFO:root:Epoch[1] Time cost=0.431
INFO:root:Epoch[1] Validation-accuracy=0.542292
INFO:root:Epoch[2] Batch [64]	Speed: 31469.63 samples/sec	accuracy=0.553846
INFO:root:Epoch[2] Batch [128]	Speed: 36495.14 samples/sec	accuracy=0.553467
INFO:root:Epoch[2] Batch [192]	Speed: 37425.32 samples/sec	accuracy=0.552002
INFO:root:Epoch[2] Train-accuracy=0.555444
INFO:root:Epoch

INFO:root:Epoch[22] Validation-accuracy=0.542292
INFO:root:Epoch[23] Batch [64]	Speed: 35298.39 samples/sec	accuracy=0.553846
INFO:root:Epoch[23] Batch [128]	Speed: 37957.08 samples/sec	accuracy=0.553467
INFO:root:Epoch[23] Batch [192]	Speed: 37220.02 samples/sec	accuracy=0.552002
INFO:root:Epoch[23] Train-accuracy=0.555444
INFO:root:Epoch[23] Time cost=0.396
INFO:root:Epoch[23] Validation-accuracy=0.542292
INFO:root:Epoch[24] Batch [64]	Speed: 32386.01 samples/sec	accuracy=0.553846
INFO:root:Epoch[24] Batch [128]	Speed: 36382.77 samples/sec	accuracy=0.553467
INFO:root:Epoch[24] Batch [192]	Speed: 36792.30 samples/sec	accuracy=0.552002
INFO:root:Epoch[24] Train-accuracy=0.555444
INFO:root:Epoch[24] Time cost=0.410
INFO:root:Epoch[24] Validation-accuracy=0.542292
INFO:root:Epoch[25] Batch [64]	Speed: 31679.76 samples/sec	accuracy=0.553846
INFO:root:Epoch[25] Batch [128]	Speed: 36465.24 samples/sec	accuracy=0.553467
INFO:root:Epoch[25] Batch [192]	Speed: 36657.19 samples/sec	accuracy=0.5

INFO:root:Epoch[45] Train-accuracy=0.555444
INFO:root:Epoch[45] Time cost=0.396
INFO:root:Epoch[45] Validation-accuracy=0.542292
INFO:root:Epoch[46] Batch [64]	Speed: 34739.02 samples/sec	accuracy=0.553846
INFO:root:Epoch[46] Batch [128]	Speed: 36957.88 samples/sec	accuracy=0.553467
INFO:root:Epoch[46] Batch [192]	Speed: 36481.04 samples/sec	accuracy=0.552002
INFO:root:Epoch[46] Train-accuracy=0.555444
INFO:root:Epoch[46] Time cost=0.404
INFO:root:Epoch[46] Validation-accuracy=0.542292
INFO:root:Epoch[47] Batch [64]	Speed: 32400.79 samples/sec	accuracy=0.553846
INFO:root:Epoch[47] Batch [128]	Speed: 36568.01 samples/sec	accuracy=0.553467
INFO:root:Epoch[47] Batch [192]	Speed: 36507.40 samples/sec	accuracy=0.552002
INFO:root:Epoch[47] Train-accuracy=0.555444
INFO:root:Epoch[47] Time cost=0.414
INFO:root:Epoch[47] Validation-accuracy=0.542292
INFO:root:Epoch[48] Batch [64]	Speed: 31765.16 samples/sec	accuracy=0.553846
INFO:root:Epoch[48] Batch [128]	Speed: 37147.03 samples/sec	accuracy=0

INFO:root:Epoch[68] Batch [192]	Speed: 38356.23 samples/sec	accuracy=0.552002
INFO:root:Epoch[68] Train-accuracy=0.555444
INFO:root:Epoch[68] Time cost=0.412
INFO:root:Epoch[68] Validation-accuracy=0.542292
INFO:root:Epoch[69] Batch [64]	Speed: 33824.03 samples/sec	accuracy=0.553846
INFO:root:Epoch[69] Batch [128]	Speed: 36098.91 samples/sec	accuracy=0.553467
INFO:root:Epoch[69] Batch [192]	Speed: 36126.39 samples/sec	accuracy=0.552002
INFO:root:Epoch[69] Train-accuracy=0.555444
INFO:root:Epoch[69] Time cost=0.410
INFO:root:Epoch[69] Validation-accuracy=0.542292
INFO:root:Epoch[70] Batch [64]	Speed: 32572.98 samples/sec	accuracy=0.553846
INFO:root:Epoch[70] Batch [128]	Speed: 36282.57 samples/sec	accuracy=0.553467
INFO:root:Epoch[70] Batch [192]	Speed: 36163.73 samples/sec	accuracy=0.552002
INFO:root:Epoch[70] Train-accuracy=0.555444
INFO:root:Epoch[70] Time cost=0.413
INFO:root:Epoch[70] Validation-accuracy=0.542292
INFO:root:Epoch[71] Batch [64]	Speed: 32432.78 samples/sec	accuracy=0

INFO:root:Epoch[91] Batch [128]	Speed: 36945.48 samples/sec	accuracy=0.553467
INFO:root:Epoch[91] Batch [192]	Speed: 37446.37 samples/sec	accuracy=0.552002
INFO:root:Epoch[91] Train-accuracy=0.555444
INFO:root:Epoch[91] Time cost=0.408
INFO:root:Epoch[91] Validation-accuracy=0.542292
INFO:root:Epoch[92] Batch [64]	Speed: 32001.31 samples/sec	accuracy=0.553846
INFO:root:Epoch[92] Batch [128]	Speed: 36454.18 samples/sec	accuracy=0.553467
INFO:root:Epoch[92] Batch [192]	Speed: 36046.27 samples/sec	accuracy=0.552002
INFO:root:Epoch[92] Train-accuracy=0.555444
INFO:root:Epoch[92] Time cost=0.416
INFO:root:Epoch[92] Validation-accuracy=0.542292
INFO:root:Epoch[93] Batch [64]	Speed: 32760.44 samples/sec	accuracy=0.553846
INFO:root:Epoch[93] Batch [128]	Speed: 37058.57 samples/sec	accuracy=0.553467
INFO:root:Epoch[93] Batch [192]	Speed: 37248.67 samples/sec	accuracy=0.552002
INFO:root:Epoch[93] Train-accuracy=0.555444
INFO:root:Epoch[93] Time cost=0.405
INFO:root:Epoch[93] Validation-accuracy=

...

INFO:root:Epoch[225] Train-accuracy=1.000000
INFO:root:Epoch[225] Time cost=0.411
INFO:root:Epoch[225] Validation-accuracy=0.758542
INFO:root:Epoch[226] Batch [64]	Speed: 36141.59 samples/sec	accuracy=0.999760
INFO:root:Epoch[226] Batch [128]	Speed: 36040.30 samples/sec	accuracy=0.999756
INFO:root:Epoch[226] Batch [192]	Speed: 35703.92 samples/sec	accuracy=1.000000
INFO:root:Epoch[226] Train-accuracy=1.000000
INFO:root:Epoch[226] Time cost=0.404
INFO:root:Epoch[226] Validation-accuracy=0.756042
INFO:root:Epoch[227] Batch [64]	Speed: 34054.68 samples/sec	accuracy=0.999760
INFO:root:Epoch[227] Batch [128]	Speed: 37077.44 samples/sec	accuracy=0.999756
INFO:root:Epoch[227] Batch [192]	Speed: 36656.25 samples/sec	accuracy=0.999756
INFO:root:Epoch[227] Train-accuracy=1.000000
INFO:root:Epoch[227] Time cost=0.412
INFO:root:Epoch[227] Validation-accuracy=0.757500
INFO:root:Epoch[228] Batch [64]	Speed: 36149.50 samples/sec	accuracy=0.999760
INFO:root:Epoch[228] Batch [128]	Speed: 35560.40 sampl

INFO:root:Epoch[248] Batch [64]	Speed: 36959.71 samples/sec	accuracy=1.000000
INFO:root:Epoch[248] Batch [128]	Speed: 35687.38 samples/sec	accuracy=1.000000
INFO:root:Epoch[248] Batch [192]	Speed: 35911.02 samples/sec	accuracy=1.000000
INFO:root:Epoch[248] Train-accuracy=1.000000
INFO:root:Epoch[248] Time cost=0.401
INFO:root:Epoch[248] Validation-accuracy=0.752917
INFO:root:Epoch[249] Batch [64]	Speed: 35868.36 samples/sec	accuracy=1.000000
INFO:root:Epoch[249] Batch [128]	Speed: 36712.97 samples/sec	accuracy=1.000000
INFO:root:Epoch[249] Batch [192]	Speed: 36213.97 samples/sec	accuracy=1.000000
INFO:root:Epoch[249] Train-accuracy=1.000000
INFO:root:Epoch[249] Time cost=0.401
INFO:root:Epoch[249] Validation-accuracy=0.754375
INFO:root:Epoch[250] Batch [64]	Speed: 30594.15 samples/sec	accuracy=1.000000
INFO:root:Epoch[250] Batch [128]	Speed: 29757.91 samples/sec	accuracy=1.000000
INFO:root:Epoch[250] Batch [192]	Speed: 36661.88 samples/sec	accuracy=1.000000
INFO:root:Epoch[250] Train-a

INFO:root:Epoch[270] Batch [192]	Speed: 36141.97 samples/sec	accuracy=1.000000
INFO:root:Epoch[270] Train-accuracy=1.000000
INFO:root:Epoch[270] Time cost=0.412
INFO:root:Epoch[270] Validation-accuracy=0.754167
INFO:root:Epoch[271] Batch [64]	Speed: 37400.80 samples/sec	accuracy=1.000000
INFO:root:Epoch[271] Batch [128]	Speed: 36069.81 samples/sec	accuracy=1.000000
INFO:root:Epoch[271] Batch [192]	Speed: 36763.95 samples/sec	accuracy=1.000000
INFO:root:Epoch[271] Train-accuracy=1.000000
INFO:root:Epoch[271] Time cost=0.395
INFO:root:Epoch[271] Validation-accuracy=0.755833
INFO:root:Epoch[272] Batch [64]	Speed: 37075.12 samples/sec	accuracy=1.000000
INFO:root:Epoch[272] Batch [128]	Speed: 36837.74 samples/sec	accuracy=1.000000
INFO:root:Epoch[272] Batch [192]	Speed: 36411.30 samples/sec	accuracy=1.000000
INFO:root:Epoch[272] Train-accuracy=1.000000
INFO:root:Epoch[272] Time cost=0.401
INFO:root:Epoch[272] Validation-accuracy=0.755208
INFO:root:Epoch[273] Batch [64]	Speed: 29719.82 sampl

INFO:root:Epoch[292] Time cost=0.401
INFO:root:Epoch[292] Validation-accuracy=0.753958
INFO:root:Epoch[293] Batch [64]	Speed: 35077.42 samples/sec	accuracy=1.000000
INFO:root:Epoch[293] Batch [128]	Speed: 35468.04 samples/sec	accuracy=1.000000
INFO:root:Epoch[293] Batch [192]	Speed: 36390.93 samples/sec	accuracy=1.000000
INFO:root:Epoch[293] Train-accuracy=1.000000
INFO:root:Epoch[293] Time cost=0.408
INFO:root:Epoch[293] Validation-accuracy=0.754583
INFO:root:Epoch[294] Batch [64]	Speed: 35706.44 samples/sec	accuracy=1.000000
INFO:root:Epoch[294] Batch [128]	Speed: 36944.37 samples/sec	accuracy=1.000000
INFO:root:Epoch[294] Batch [192]	Speed: 37193.59 samples/sec	accuracy=1.000000
INFO:root:Epoch[294] Train-accuracy=1.000000
INFO:root:Epoch[294] Time cost=0.396
INFO:root:Epoch[294] Validation-accuracy=0.753958
INFO:root:Epoch[295] Batch [64]	Speed: 34501.75 samples/sec	accuracy=1.000000
INFO:root:Epoch[295] Batch [128]	Speed: 35515.48 samples/sec	accuracy=1.000000
INFO:root:Epoch[295]

Accuracy: 75.375 %


In [20]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'DDNN_SVM'
epoch = 300
mod.save_checkpoint(prefix, epoch)

INFO:root:Saved checkpoint to "DDNN_SVM-0300.params"


In [21]:
modelMetric(mod, 'Finish predict of DDNN + SVM...')

INFO:root:Finish predict of DDNN + SVM...
INFO:root:Test accuracy = 0.754500


             precision    recall  f1-score   support

        0.0       0.77      0.78      0.78      2590
        1.0       0.74      0.72      0.73      2188

avg / total       0.75      0.75      0.75      4778

[[2021  569]
 [ 604 1584]]


## 2.4 Inception Batach Normalization

In [22]:
# Reference: www.mxnet.io
# Basic Conv + BN + ReLU factory
def ConvFactory(data, num_filter, kernel, stride=(1,1), pad=(0, 0), act_type="relu"):
    conv = mx.symbol.Convolution(data=data, workspace=256,
                                 num_filter=num_filter, kernel=kernel, stride=stride, pad=pad)
    bn = mx.symbol.BatchNorm(data=conv)
    act = mx.symbol.Activation(data = bn, act_type=act_type)
    return act

# A Simple Downsampling Factory
def DownsampleFactory(data, ch_3x3):
    conv = ConvFactory(data=data, kernel=(3, 3), stride=(2, 2), num_filter=ch_3x3, pad=(1, 1))
    pool = mx.symbol.Pooling(data=data, kernel=(3, 3), stride=(2, 2),pad=(1,1), pool_type='max')
    concat = mx.symbol.Concat(*[conv, pool])
    return concat

# A Simple module
def SimpleFactory(data, ch_1x1, ch_3x3):
    conv1x1 = ConvFactory(data=data, kernel=(1, 1), pad=(0, 0), num_filter=ch_1x1)
    conv3x3 = ConvFactory(data=data, kernel=(3, 3), pad=(1, 1), num_filter=ch_3x3)
    concat = mx.symbol.Concat(*[conv1x1, conv3x3])
    return concat

def InceptionFactory(data, num_1x1, num_3x3red, num_3x3, num_d5x5red, num_d5x5, pool, proj, name):
    c1x1 = ConvFactoryB(data=data, num_filter=num_1x1, kernel=(1, 1), name=('%s_1x1' % name))
    c3x3r = ConvFactoryB(data=data, num_filter=num_3x3red, kernel=(1, 1), name=('%s_3x3' % name), suffix='_reduce')
    c3x3 = ConvFactoryB(data=c3x3r, num_filter=num_3x3, kernel=(3, 3), pad=(1, 1), name=('%s_3x3' % name))
    cd5x5r = ConvFactoryB(data=data, num_filter=num_d5x5red, kernel=(1, 1), name=('%s_double_3x3' % name), suffix='_reduce')
    cd5x5 = ConvFactoryB(data=cd5x5r, num_filter=num_d5x5, kernel=(5, 5), pad=(2, 2), name=('%s_double_3x3_1' % name))
    pooling = mx.symbol.Pooling(data=data, kernel=(3, 3), stride=(1, 1), pad=(1, 1), pool_type=pool, name=('%s_pool_%s_pool' % (pool, name)))
    cproj = ConvFactoryB(data=pooling, num_filter=proj, kernel=(1, 1), name=('%s_proj' %  name))
    concat = mx.symbol.Concat(*[c1x1, c3x3, cd5x5, cproj], name='ch_concat_%s_chconcat' % name)
    return concat

In [23]:
# Inception Batch Normalization Network
#https://arxiv.org/pdf/1502.03167.pdf
data = mx.sym.Variable('data')

conv1 = ConvFactory(data=data, kernel=(3,3), pad=(1,1), num_filter=96, act_type="relu")
in3a = SimpleFactory(conv1, 32, 32)
in3b = SimpleFactory(in3a, 32, 48)
in3c = DownsampleFactory(in3b, 80)
in4a = SimpleFactory(in3c, 112, 48)
in4b = SimpleFactory(in4a, 96, 64)
in4c = SimpleFactory(in4b, 80, 80)
in4d = SimpleFactory(in4c, 48, 96)
in4e = DownsampleFactory(in4d, 96)
in5a = SimpleFactory(in4e, 176, 160)
in5b = SimpleFactory(in5a, 176, 160)
pool = mx.symbol.Pooling(data=in5b, pool_type="avg", kernel=(7,7), name="global_pool")
flatten = mx.symbol.Flatten(data=pool, name="flatten1")
fc = mx.symbol.FullyConnected(data=flatten, num_hidden=62, name="fc1")
ibn = mx.symbol.SVMOutput(data=fc, name='svm')

# draw network chart
shape = {"data" : (batch_size, 1, 32, 32)}
mx.viz.plot_network(symbol=ibn, shape=shape)
network_chart=mx.viz.plot_network(symbol=ibn, shape=shape)
network_chart.format = 'png'
network_chart.render('Inception Batch Normalization')

'Inception Batch Normalization.png'

<img src = 'Inception Batch Normalization.png'>

In [24]:
mod = mx.mod.Module(
    context = mx.gpu(0),  # Run on GPU 0
    symbol = ibn,         # Use the network we just defined
    label_names = ['svm_label'],
)
mod.fit(
    train_data=train_iters,
    eval_data=val_iters,  
    batch_end_callback = mx.callback.Speedometer(batch_size, batch_size),  
    num_epoch = 50,       # Train for 100 epochs
    optimizer_params = {
        'learning_rate': 0.001,  # Learning rate
        'momentum': 0.9,       # Momentum for SGD with momentum
        'wd': 0.00001,         # Weight decay for regularization
    },
)

print('Accuracy:', mod.score(val_iters, mx.metric.Accuracy())[0][1]*100, '%')

INFO:root:Epoch[0] Batch [64]	Speed: 1146.57 samples/sec	accuracy=0.592308
INFO:root:Epoch[0] Batch [128]	Speed: 1110.26 samples/sec	accuracy=0.657227
INFO:root:Epoch[0] Batch [192]	Speed: 1211.53 samples/sec	accuracy=0.704590
INFO:root:Epoch[0] Train-accuracy=0.710685
INFO:root:Epoch[0] Time cost=12.373
INFO:root:Epoch[0] Validation-accuracy=0.734583
INFO:root:Epoch[1] Batch [64]	Speed: 1211.97 samples/sec	accuracy=0.736058
INFO:root:Epoch[1] Batch [128]	Speed: 1211.69 samples/sec	accuracy=0.740967
INFO:root:Epoch[1] Batch [192]	Speed: 1202.80 samples/sec	accuracy=0.754150
INFO:root:Epoch[1] Train-accuracy=0.746472
INFO:root:Epoch[1] Time cost=11.837
INFO:root:Epoch[1] Validation-accuracy=0.726667
INFO:root:Epoch[2] Batch [64]	Speed: 1207.22 samples/sec	accuracy=0.775721
INFO:root:Epoch[2] Batch [128]	Speed: 1202.65 samples/sec	accuracy=0.776855
INFO:root:Epoch[2] Batch [192]	Speed: 1204.64 samples/sec	accuracy=0.791992
INFO:root:Epoch[2] Train-accuracy=0.785786
INFO:root:Epoch[2] Tim

INFO:root:Epoch[23] Batch [64]	Speed: 1197.48 samples/sec	accuracy=0.984856
INFO:root:Epoch[23] Batch [128]	Speed: 1199.46 samples/sec	accuracy=0.983643
INFO:root:Epoch[23] Batch [192]	Speed: 1199.74 samples/sec	accuracy=0.983887
INFO:root:Epoch[23] Train-accuracy=0.982359
INFO:root:Epoch[23] Time cost=11.926
INFO:root:Epoch[23] Validation-accuracy=0.738958
INFO:root:Epoch[24] Batch [64]	Speed: 1132.90 samples/sec	accuracy=0.984375
INFO:root:Epoch[24] Batch [128]	Speed: 1195.24 samples/sec	accuracy=0.987061
INFO:root:Epoch[24] Batch [192]	Speed: 1200.93 samples/sec	accuracy=0.986572
INFO:root:Epoch[24] Train-accuracy=0.984375
INFO:root:Epoch[24] Time cost=12.133
INFO:root:Epoch[24] Validation-accuracy=0.749792
INFO:root:Epoch[25] Batch [64]	Speed: 1197.35 samples/sec	accuracy=0.985817
INFO:root:Epoch[25] Batch [128]	Speed: 1199.57 samples/sec	accuracy=0.988281
INFO:root:Epoch[25] Batch [192]	Speed: 1199.85 samples/sec	accuracy=0.986328
INFO:root:Epoch[25] Train-accuracy=0.981855
INFO:r

INFO:root:Epoch[45] Time cost=11.987
INFO:root:Epoch[45] Validation-accuracy=0.739583
INFO:root:Epoch[46] Batch [64]	Speed: 1172.46 samples/sec	accuracy=0.998077
INFO:root:Epoch[46] Batch [128]	Speed: 1196.41 samples/sec	accuracy=0.998047
INFO:root:Epoch[46] Batch [192]	Speed: 1199.61 samples/sec	accuracy=0.999023
INFO:root:Epoch[46] Train-accuracy=0.999496
INFO:root:Epoch[46] Time cost=12.010
INFO:root:Epoch[46] Validation-accuracy=0.740833
INFO:root:Epoch[47] Batch [64]	Speed: 1198.60 samples/sec	accuracy=0.997837
INFO:root:Epoch[47] Batch [128]	Speed: 1165.33 samples/sec	accuracy=0.998779
INFO:root:Epoch[47] Batch [192]	Speed: 1199.21 samples/sec	accuracy=0.998779
INFO:root:Epoch[47] Train-accuracy=0.997480
INFO:root:Epoch[47] Time cost=12.024
INFO:root:Epoch[47] Validation-accuracy=0.739167
INFO:root:Epoch[48] Batch [64]	Speed: 1202.05 samples/sec	accuracy=0.997356
INFO:root:Epoch[48] Batch [128]	Speed: 1200.94 samples/sec	accuracy=0.996826
INFO:root:Epoch[48] Batch [192]	Speed: 11

Accuracy: 73.75 %


In [25]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'Inception_BN'
epoch = 50
mod.save_checkpoint(prefix, epoch)

INFO:root:Saved checkpoint to "Inception_BN-0050.params"


In [26]:
modelMetric(mod, 'Finish predict of Inception BN..')

INFO:root:Finish predict of Inception BN..
INFO:root:Test accuracy = 0.737547


             precision    recall  f1-score   support

        0.0       0.75      0.78      0.76      2590
        1.0       0.73      0.68      0.70      2188

avg / total       0.74      0.74      0.74      4778

[[2031  559]
 [ 695 1493]]
